# Call Context Summary

## 1. Setup Workspace

In [9]:
import json
import pathlib
import copy

import anytree

import paptree

## 2. Load Data

In [10]:
trace_file = pathlib.Path.cwd().parent / "data/fibonacci/paptrace.json"
trees = paptree.utils.from_file(trace_file)
print(f"Loaded {len(trees)} traces.")

Loaded 36 traces.


In [50]:
# Take a peek at a few of the traces for the naive recursive algorithm.
for tree in trees[:4]:
    print(anytree.RenderTree(tree), "\n")

CallNode(name=2106190, params=[{'name': 'n', 'value': '0'}], sig='unsigned long long fibonacci::RecursiveNaive(unsigned short)', type='CalleeExpr')
└── StmtNode(desc='n < 2', name=2106009, type='IfThenStmt')
    └── StmtNode(desc='return n', name=2106007, type='ReturnStmt') 

CallNode(name=2106190, params=[{'name': 'n', 'value': '1'}], sig='unsigned long long fibonacci::RecursiveNaive(unsigned short)', type='CalleeExpr')
└── StmtNode(desc='n < 2', name=2106009, type='IfThenStmt')
    └── StmtNode(desc='return n', name=2106007, type='ReturnStmt') 

CallNode(name=2106190, params=[{'name': 'n', 'value': '2'}], sig='unsigned long long fibonacci::RecursiveNaive(unsigned short)', type='CalleeExpr')
└── StmtNode(desc='return RecursiveNaive(n - 1) + RecursiveNaive(n - 2)', name=2106188, type='ReturnStmt')
    └── StmtNode(desc='unsigned long long', name=2106184, type='+')
        └── StmtNode(desc='int', name=2106117, type='-')
            ├── CallNode(name=2106190, params=[{'name': 'n', 'valu

In [51]:
# Take a peek at a few of the traces for the iterative algorithm.
for tree in trees[18:22]:
    print(anytree.RenderTree(tree), "\n")

CallNode(name=2110045, params=[{'name': 'n', 'value': '0'}], sig='unsigned long long fibonacci::Iterative(unsigned short)', type='CalleeExpr')
└── StmtNode(desc='n < 2', name=2109766, type='IfThenStmt')
    └── StmtNode(desc='return n', name=2109764, type='ReturnStmt') 

CallNode(name=2110045, params=[{'name': 'n', 'value': '1'}], sig='unsigned long long fibonacci::Iterative(unsigned short)', type='CalleeExpr')
└── StmtNode(desc='n < 2', name=2109766, type='IfThenStmt')
    └── StmtNode(desc='return n', name=2109764, type='ReturnStmt') 

CallNode(name=2110045, params=[{'name': 'n', 'value': '2'}], sig='unsigned long long fibonacci::Iterative(unsigned short)', type='CalleeExpr')
├── StmtNode(desc='for (unsigned short i = 2; i <= n; ++i)', name=2110029, type='ForStmt')
│   ├── StmtNode(desc='LoopIter', name=2110029, type='LoopIter')
│   │   ├── StmtNode(desc='unsigned long long', name=2109990, type='=')
│   │   │   └── StmtNode(desc='unsigned long long', name=2109986, type='+')
│   │   ├

## 3. Extract Call Nodes

In [71]:
call_nodes = []
for tree in trees:
    call_nodes.extend(
        anytree.findall(tree.root, filter_=lambda n: isinstance(n, paptree.CallNode)))

print(f"Number of call nodes: {len(call_nodes)}")

Number of call nodes: 288


In [72]:
print(f"Unique call nodes: {len(set([str(n) for n in call_nodes]))}")

Unique call nodes: 184


In [73]:
sigs = set([node.sig for node in call_nodes])
print(f"Number of unique signatures: {len(sigs)}")
display(sigs)

Number of unique signatures: 6


{'reference std::vector<unsigned long long>::operator[](size_type)',
 'unsigned long long fibonacci::(anonymous namespace)::RecursiveMemoImpl(unsigned short, std::vector<unsigned long long> &)',
 'unsigned long long fibonacci::Iterative(unsigned short)',
 'unsigned long long fibonacci::LookupTable(unsigned short)',
 'unsigned long long fibonacci::RecursiveMemo(unsigned short)',
 'unsigned long long fibonacci::RecursiveNaive(unsigned short)'}

### 3.1. Caller Nodes

Caller nodes represent a call to an uninstrumented function. The complexity of these nodes must be provided by the user since there is no trace data to analyze.

In [97]:
caller_nodes = [node for node in call_nodes if node.type == "CallerExpr"]
print(f"Number of caller nodes: {len(caller_nodes)}")

caller_node_sigs = set([node.sig for node in caller_nodes])
print(f"Number of unique caller sigs: {len(caller_node_sigs)}")
print("Unique caller node sigs:")
print("\n".join(caller_node_sigs))

Number of caller nodes: 110
Number of unique caller sigs: 1
Unique caller node sigs:
reference std::vector<unsigned long long>::operator[](size_type)


### 3.2. Callee Nodes

Callee nodes represent a call to an instrumented function. The complexity of these nodes can be deduced from the collected trace data, assuming that a user has provided the complexity for any dependent caller nodes.

In [98]:
callee_nodes = [node for node in call_nodes if node.type == "CalleeExpr"]
print(f"Number of callee nodes: {len(callee_nodes)}")

callee_node_sigs = set([node.sig for node in callee_nodes])
print(f"Number of unique callee sigs: {len(callee_node_sigs)}")
print("Unique callee node sigs:")
print("\n".join(callee_node_sigs))

Number of callee nodes: 178
Number of unique callee sigs: 5
Unique callee node sigs:
unsigned long long fibonacci::LookupTable(unsigned short)
unsigned long long fibonacci::Iterative(unsigned short)
unsigned long long fibonacci::RecursiveNaive(unsigned short)
unsigned long long fibonacci::RecursiveMemo(unsigned short)
unsigned long long fibonacci::(anonymous namespace)::RecursiveMemoImpl(unsigned short, std::vector<unsigned long long> &)


## 5. Summarize Call Contexts

Of the 288 call nodes, only 6 unique signatures were detected. How many contexts are there per signature? How many unique contexts per signature?

In [99]:
call_map = {}
for call_node in unique_call_nodes:
    id = call_node["id"]
    if not id in call_map:
        call_map[id] = {"sig": call_node["sig"], "ctxs": {}}
    ctx = str(call_node["params"])
    if not ctx in call_map[id]["ctxs"]:
        call_map[id]["ctxs"][ctx] = len(call_map[id]["ctxs"])
    else:
        raise Exception(f"Duplicate call node ctx: {ctx}!")
        
print(f"Number of top-level call map entries: {len(call_map)}")
display(call_map)

NameError: name 'unique_call_nodes' is not defined